In [ ]:
!pip install -r requirements.txt

### Import thư viện

In [3]:
import cv2
import numpy as np
from PIL import Image
from torchvision import datasets
import torchvision.transforms as transforms
import torch
from torch import nn

In [4]:
print(torch.__version__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

2.1.1+cu121
cuda


In [9]:
# import mediapipe as mp

# mp_hands = mp.solutions.hands
# mp_drawing = mp.solutions.drawing_utils 
# mp_drawing_styles = mp.solutions.drawing_styles
# hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.9)

In [10]:
# class_names = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
#                       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
#                       'del', 'nothing', 'space'])

# data_transform = transforms.Compose([
#                 transforms.Resize(size = (128, 128)),         # Resize our images to 224x224
#                 transforms.ToTensor()
#             ])

### Khai báo đường dẫn

In [7]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

# Create model save path
MODEL_NAME = "ASL_HandSignLang_EfficientNetB0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

### Khai báo kiến trúc của Efficientnet-b0

In [11]:
from efficientnet_pytorch import EfficientNet
from torchinfo import summary

num_classes = len(class_names)    # Number of classes

# Instantiate the EfficientNet model
model = EfficientNet.from_pretrained('efficientnet-b0').to(device)
model._fc = nn.Linear(model._fc.in_features, num_classes)  

summary(model=model,
        input_size=(192, 3, 128, 128),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Loaded pretrained weights for efficientnet-b0


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [192, 3, 128, 128]   [192, 29]            --                   True
├─Conv2dStaticSamePadding (_conv_stem)                       [192, 3, 128, 128]   [192, 32, 64, 64]    864                  True
│    └─ZeroPad2d (static_padding)                            [192, 3, 128, 128]   [192, 3, 129, 129]   --                   --
├─BatchNorm2d (_bn0)                                         [192, 32, 64, 64]    [192, 32, 64, 64]    64                   True
├─MemoryEfficientSwish (_swish)                              [192, 32, 64, 64]    [192, 32, 64, 64]    --                   --
├─ModuleList (_blocks)                                       --                   --                   --                   True
│    └─MBConvBlock (0)                                       [192, 32, 64, 64]    [192, 16, 64, 

### Load pretrained model

In [12]:
loaded_model = EfficientNet.from_pretrained('efficientnet-b0').to('cpu')
loaded_model._fc = nn.Linear(model._fc.in_features, 29)

loaded_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

# print(f"Loaded model:\n{loaded_model}")
print(f"Model on device:\n{next(loaded_model.parameters()).device}")

Loaded pretrained weights for efficientnet-b0
Model on device:
cpu


### Real-time Testing

In [13]:
def make_prediction(model, data, device):
  pred_probs = []
  model.to(device)
  model.eval()
  with torch.inference_mode():
      sample = data.to(device)  
      pred_logit = model(sample.unsqueeze(0))  
      pred_prob = torch.softmax(pred_logit.squeeze(), dim=0) 
      pred_probs.append(pred_prob.cpu())

  return torch.stack(pred_probs)

In [14]:
def label_extraction(model, img, device='cpu'):   
    transformed_img = data_transform(img)

    loaded_pred_prob = make_prediction(model,
                            transformed_img,
                            device)

    loaded_pred_class = loaded_pred_prob.argmax(dim = 1)
    return class_names[loaded_pred_class]

In [15]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8) as hands:
    while cap.isOpened():
        data_aux = []
        x_ = []
        y_ = []

        ret, frame = cap.read()
        H, W, _ = frame.shape

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = cv2.flip(frame_rgb, 1)
        frame_rgb.flags.writeable = False
        results = hands.process(frame_rgb)
        frame_rgb.flags.writeable = True
        frame_rgb = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        
        frame = Image.fromarray(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)
                    x_.append(x)
                    y_.append(y)

            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10

            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            # Ký hiệu được xuất ra
            # prediction = model.predict([np.array(data_aux)[0:42]])[0]
            
            prediction = label_extraction(loaded_model, frame)

            # Hiển thị bounding box và dự đoán ký hiệu
            cv2.rectangle(frame_rgb, (x1, y1 - 10), (x2, y2), (255, 99, 173), 6)
            cv2.putText(frame_rgb, prediction, (x1, y1), cv2.FONT_HERSHEY_DUPLEX, 5, (255, 0, 0), 5, cv2.LINE_AA)

        cv2.imshow('frame', frame_rgb)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [5]:
import cv2
import numpy as np
from PIL import Image
from torchvision import datasets
import torchvision.transforms as transforms
import torch
from torch import nn

from efficientnet_pytorch import EfficientNet
from torchinfo import summary

class_names = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
                      'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                      'del', 'nothing', 'space'])

data_transform = transforms.Compose([
                transforms.Resize(size = (128, 128)),         # Resize our images to 224x224
                transforms.ToTensor()
            ])


def create_model():
    model = EfficientNet.from_pretrained('efficientnet-b0')
    model._fc = nn.Linear(model._fc.in_features, len(class_names))
    model.load_state_dict(torch.load(f="./models/ASL_HandSignLang_EfficientNetB0.pth", map_location=torch.device('cpu')))
    model.eval()
    return model

def predict(model, img):
    img = data_transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(img)
    result = {}
    for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
        prob = torch.softmax(outputs, dim=1)[0, idx].item()
        result[class_names[idx]] = prob*100
        print('{label:<75} ({p:.2f}%)'.format(label=class_names[idx], p=prob*100))
    return result

In [6]:
model = create_model()

Loaded pretrained weights for efficientnet-b0


In [8]:
image = Image.open("asl-dataset\Test\A_test.jpg")
result = predict(model, image)

A                                                                           (100.00%)
T                                                                           (0.00%)
Y                                                                           (0.00%)
M                                                                           (0.00%)
S                                                                           (0.00%)


In [16]:
def predict(model, img, device='cpu'):
    transformed_img = data_transform(img)

    model.to(device)
    
    with torch.inference_mode():
        sample = transformed_img.to(device).unsqueeze(0)
        
        pred_logit = model(sample)
        pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)
        
        loaded_pred_class = pred_prob.argmax().item()
        
    return class_names[loaded_pred_class]

In [24]:
image = Image.open("asl-dataset\Test/3004.jpg")
result = predict(model, image)
result

'P'

In [ ]:
def label_extraction(model, img, device='cpu'):   
    transformed_img = data_transform(img)

    loaded_pred_prob = make_prediction(model,
                            transformed_img,
                            device)

    loaded_pred_class = loaded_pred_prob.argmax(dim = 1)
    return class_names[loaded_pred_class]